<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/main/GS_v0_2_con_vocab_senado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:

import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
scraps-pdfs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [39]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [40]:
import pandas as pd
base=pd.read_csv('/content/base_formateada.csv')

<ipython-input-40-71200aca493d>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base=pd.read_csv('/content/base_formateada.csv')


In [41]:
base.Legislatura.unique()

array([10, 11, 12, 13, 14, '14', '1/', '2/', '3/', '4/', '5/', '6/', '7/',
       '8/', '9/'], dtype=object)

In [42]:
# Crear un diccionario de mapeo
mapeo = {
    '1/': 'I', '2/': 'II', '3/': 'III', '4/': 'IV', '5/': 'V', '6/': 'VI',
    '7/': 'VII', '8/': 'VIII', '9/': 'IX', 10: 'X', 11: 'XI',
    12: 'XII', 13: 'XIII', 14: 'XIV', '14': 'XIV'
}

In [45]:
base['Legislatura']=base['Legislatura'].replace(mapeo)

In [46]:
import pickle

with open('/content/vocabulary_senado.voc', 'rb') as f:
    inter = pickle.load(f)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2) ,vocabulary=inter)
matrix = vect.fit_transform(base['tokens'])

In [48]:
len(vect.vocabulary_)

176701

In [49]:
# Esta celda tarda unos 32 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=30, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, n_components=30,
                          random_state=0)

In [50]:
import numpy as np
from numpy import argsort
import statistics
from statistics import mode
from scipy.special import logsumexp
import scipy as sp

# Topics por weights

In [51]:
disc_weights=matrix*lda.components_.transpose()

In [52]:
base['Topic_weight']=disc_weights.argmax(axis=1)+1

In [53]:
base.fecha=pd.to_datetime(base.fecha_formato, infer_datetime_format=True)

In [54]:
# Por LEGISLATURA por WEIGHTS
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.Legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_weight'].value_counts().sort_index()/base_legislatura['Topic_weight'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_weights_senado.csv')

In [55]:
# Por AÑO por weights
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha_formato).year==x]
  topics_por_año[x]=base_year['Topic_weight'].value_counts().sort_index()/base_year['Topic_weight'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_weights_senado.csv')

# FREX

In [56]:
def frex(mod, w=0.7):
    log_beta = np.log(mod.components_)
    log_exclusivity = log_beta - logsumexp(log_beta, axis=0)
    exclusivity_ecdf = np.apply_along_axis(ecdf, 1, log_exclusivity)
    freq_ecdf = np.apply_along_axis(ecdf, 1, log_beta)
    out = 1. / (w / exclusivity_ecdf + (1 - w) / freq_ecdf)
    return out

def ecdf(arr):
    return sp.stats.rankdata(arr, method='max') / arr.size

def t_FREX(row):
  return frex_df[matrix[row['id']].argmax()].argsort()[9]

In [57]:
frex_df=pd.DataFrame(frex(lda))

In [58]:
# Se crea una columna con el topic que se asigna por frex a cada discurso de la base
base['id']=range(0,len(base))
base['Topic_FREX']=base.apply (lambda row: t_FREX(row), axis=1)+1

In [59]:
# Por LEGISLATURA por FREX
legislaturas=['I', 'II', 'III', 'IV', 'V', 'VI', 'VII','VIII', 'IX', 'X', 'XI', 'XII','XIII']
topics_por_legislatura=pd.DataFrame()
for x in legislaturas:
  base_legislatura=base[base.Legislatura==x]
  topics_por_legislatura[x]=base_legislatura['Topic_FREX'].value_counts().sort_index()/base_legislatura['Topic_FREX'].value_counts().sort_index().sum()
topics_por_legislatura.to_csv('topics_por_legislatura_por_frex_senado.csv')

In [60]:
# Por AÑO por FREX
topics_por_año=pd.DataFrame()
for x in range (1979, 2020):
  base_year=base[pd.DatetimeIndex(base.fecha_formato).year==x]
  topics_por_año[x]=base_year['Topic_FREX'].value_counts().sort_index()/base_year['Topic_FREX'].value_counts().sort_index().sum()
topics_por_año.to_csv('topics_por_año_por_frex_senado.csv')

# Bigramas por weights

In [61]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [62]:
bigramas.to_csv('bigramas_por_weights_30topics_senado.csv')

# Bigramas por frex

In [63]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [64]:
bigramas.to_csv('bigramas_por_frex_30topics_senado.csv')

# Bigramas para 10 Topics

In [65]:
# Esta celda tarda unos 20 minutos en correr
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=50, learning_method='batch', learning_offset=50.,random_state=0)
lda.fit(matrix)

LatentDirichletAllocation(learning_offset=50.0, max_iter=50, random_state=0)

In [66]:
bigramas=pd.DataFrame()
feature_names=vect.get_feature_names_out()
no_top_words=20
for topic_idx, topic in enumerate(lda.components_):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [67]:
bigramas.to_csv('bigramas_por_weights_10topics_senado.csv')

In [68]:
"Topic %d:" % (topic_idx + 1)
for topic_idx, topic in enumerate(frex(lda)):
  bigramas["Topic %d:" % (topic_idx + 1)]=[feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]

In [69]:
bigramas.to_csv('bigramas_por_frex_10topics_senado.csv')

# Bajada

In [70]:
from google.colab import files
files.download('topics_por_legislatura_por_weights_senado.csv')
files.download('topics_por_año_por_weights_senado.csv')
files.download('topics_por_legislatura_por_frex_senado.csv')
files.download('topics_por_año_por_frex_senado.csv')
files.download('bigramas_por_weights_30topics_senado.csv')
files.download('bigramas_por_frex_30topics_senado.csv')
files.download('bigramas_por_weights_10topics_senado.csv')
files.download('bigramas_por_frex_10topics_senado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>